In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [28]:
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [29]:
dataset = pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [30]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [31]:
dataset.drop('User ID',axis=1)

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [32]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [33]:
indep = dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep = dataset['Purchased']

In [34]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep, dep, test_size=1/3,random_state=0)

In [35]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [36]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid = {'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'gamma':['scale', 'auto'],
              'decision_function_shape':['ovo', 'ovr']}
grid = GridSearchCV (SVC(), param_grid, refit = True, verbose = 3, n_jobs = -1, scoring = 'f1_weighted')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'decision_function_shape': ['ovo', 'ovr'],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [37]:
# finding the best parameter
#print(grid.best_params_)
re = grid.cv_results_
grid_predictions = grid.predict(X_test)

In [38]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,grid_predictions)

In [39]:
from sklearn.metrics import classification_report
clf_report = classification_report(Y_test,grid_predictions)

In [40]:
from sklearn.metrics import f1_score
f1_macro = f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter{'decision_function_shape': 'ovo', 'gamma': 'scale', 'kernel': 'rbf'}: 0.9111247448089108


In [41]:
print("The Confusion Matrix:\n",cm)

The Confusion Matrix:
 [[77  8]
 [ 4 45]]


In [42]:
print("classification_report:\n",clf_report)

classification_report:
               precision    recall  f1-score   support

           0       0.95      0.91      0.93        85
           1       0.85      0.92      0.88        49

    accuracy                           0.91       134
   macro avg       0.90      0.91      0.91       134
weighted avg       0.91      0.91      0.91       134



In [43]:
table = pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_decision_function_shape,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009398,0.000489,0.010999,0.001673,ovo,scale,linear,"{'decision_function_shape': 'ovo', 'gamma': 's...",0.776290,0.770070,0.698235,0.923510,0.901744,0.813970,0.085375,5
1,0.007799,0.000398,0.009798,0.000981,ovo,scale,poly,"{'decision_function_shape': 'ovo', 'gamma': 's...",0.799620,0.804628,0.705612,0.943041,0.813763,0.813333,0.075752,9
2,0.009000,0.001789,0.011598,0.002059,ovo,scale,rbf,"{'decision_function_shape': 'ovo', 'gamma': 's...",0.887346,0.886792,0.850543,0.944161,0.962264,0.906221,0.041022,1
3,0.008997,0.002608,0.009600,0.000799,ovo,scale,sigmoid,"{'decision_function_shape': 'ovo', 'gamma': 's...",0.732497,0.752940,0.626628,0.905069,0.826499,0.768727,0.093467,15
4,0.007998,0.000893,0.009400,0.001356,ovo,auto,linear,"{'decision_function_shape': 'ovo', 'gamma': 'a...",0.776290,0.770070,0.698235,0.923510,0.901744,0.813970,0.085375,5
5,0.008198,0.001165,0.009400,0.001357,ovo,auto,poly,"{'decision_function_shape': 'ovo', 'gamma': 'a...",0.799620,0.804628,0.705612,0.943041,0.813763,0.813333,0.075752,9
6,0.006999,0.000633,0.011399,0.001960,ovo,auto,rbf,"{'decision_function_shape': 'ovo', 'gamma': 'a...",0.887346,0.886792,0.850543,0.944161,0.962264,0.906221,0.041022,1
7,0.007800,0.001165,0.009400,0.000489,ovo,auto,sigmoid,"{'decision_function_shape': 'ovo', 'gamma': 'a...",0.770712,0.752940,0.626628,0.905069,0.826499,0.776370,0.091739,13
8,0.008399,0.001855,0.010400,0.001020,ovr,scale,linear,"{'decision_function_shape': 'ovr', 'gamma': 's...",0.776290,0.770070,0.698235,0.923510,0.901744,0.813970,0.085375,5
9,0.007599,0.000801,0.008999,0.000002,ovr,scale,poly,"{'decision_function_shape': 'ovr', 'gamma': 's...",0.799620,0.804628,0.705612,0.943041,0.813763,0.813333,0.075752,9


In [44]:
age_input = float(input("Age:"))
salary_input = float(input("Salary:"))
sex_male_input = int(input("sex_male 0 or 1:"))
Future_Prediction = grid.predict([[age_input, salary_input, sex_male_input]])
Future_Prediction

Age: 32
Salary: 141244
sex_male 0 or 1: 1


array([1], dtype=int64)